In [ ]:
### VAR model for Multivariate Time Series Analysis

In [ ]:
# Data import as DataFrame
import pandas as pd
EuStocks = pd.read_csv("EuIndices.csv")
EuStocks.head()

In [ ]:
# Providing the index to the time series
EuStocks_DF = pd.DataFrame(data = EuStocks.values, columns = ['0', 'DAX', 'SMI', 'CAC', 'FTSE'],
                           index = pd.DatetimeIndex(start = '1991-01-01', periods = 1860, freq = 'B'))
EuStocks_DF = EuStocks_DF.drop(columns = ['0'])
EuStocks_DF.head(20)

In [ ]:
# Creating the test for stationarity
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput) 

In [ ]:
# Loop to check all columns of the DF
import functools
EuStocks_DF.apply(functools.partial(test_stationarity))

In [ ]:
# Differencing to get the data stationary
EuStocksDiff = EuStocks_DF.diff().dropna()
EuStocksDiff.head()

In [ ]:
# Retest the differenced series
EuStocksDiff.apply(functools.partial(test_stationarity))

In [ ]:
# Creating the VAR model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(EuStocksDiff)

In [ ]:
# Best model order
modsel = model.select_order(15)
modsel.summary()

In [ ]:
# Automated model order selection
results = model.fit(maxlags=15, ic='aic')
results.summary()

In [ ]:
# Granger test for causality

grangres = results.test_causality('DAX', ['SMI', 'CAC', 'FTSE'], kind='f')
grangres.summary()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (12, 10))
results.plot_forecast(50)
plt.tight_layout(h_pad = 1.15)
plt.show()

In [ ]:
fcast = results.forecast(EuStocksDiff.values, 50)

In [ ]:
pd.DataFrame(fcast).head(10)

In [ ]:
DAXvalues = pd.DataFrame(fcast)[0]
DAXvalues.head(10)

In [ ]:
EuStocks_DF.tail(1)

In [ ]:
import numpy as np

DeDiff = np.cumsum(DAXvalues) + 5473.72

In [ ]:
plt.figure(figsize = (9,7))
plt.plot(DeDiff)
plt.title('Dax Forecast')
plt.grid()
plt.show()